# Análise dos dados

## Sobre a análise

Temos três tipos de dados neste projeto:

1. Dados de treino
2. Dados de validação
3. Itens complementares

Os dados de treino e de validação representam informações sobre os usuários do G1, falando sobre o tipo de usuário e o histórico de acesso de notícias dele.\
Já os itens complementares são dados sobre as páginas de notícias do G1: data de publicação e o conteúdo da notícia (título, subtítulo e corpo).

O objetivo neste notebook é responder algumas perguntas sobre os dados, a fim de descobrir quais tratamentos precisaremos realizar e quais features novas poderemos criar.

Ao final, iremos mostrar as conclusões que chegamos através da análise de cada um dos tipos de dados.

## Configuração

In [1]:
from pathlib import Path

import pandas as pd

Temos vários tipos de dados que são considerados strings e queremos utilizar o tipo correto para processá-los, por isso vamos utilizar o `infer_string` do Pandas. Isso irá fazer com que as strings recebam o tipo `"string[pyarrow]"`, que processa as strings de uma forma mais eficiente do que o NumPy.

In [2]:
pd.options.future.infer_string = True  # type: ignore[attr-defined]

In [3]:
import pyarrow.csv as pv


def read_csv_pyarrow(path: Path) -> pd.DataFrame:
    """Read a CSV file with PyArrow as backend.

    Args:
        path (Path): the CSV file path

    Returns:
        pd.DataFrame: a Pandas DataFrame

    """
    try:
        return pd.read_csv(
            path,
            engine="pyarrow",
            dtype_backend="pyarrow",
        )
    except pd.errors.ParserError:
        return pv.read_csv(
            path,
            parse_options=pv.ParseOptions(newlines_in_values=True),
        ).to_pandas(types_mapper=pd.ArrowDtype)

## Análise dos dados de treino

In [4]:
train_path = "../data/raw/files/treino/*.csv"
train_files = Path.cwd().glob(train_path)

train = pd.concat(
    (read_csv_pyarrow(file) for file in train_files),
    ignore_index=True,
)

train.head(2)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,acf7fbca426d323f580adddb4fa0f73598294df91cdaee...,Non-Logged,1,d6644c32-5828-4f88-ba44-a8d74c01d7bf,1657303857108,24,48232,47.5,1,1657303857108
1,0972299721e32759020a313bd97dcee19227fc3dd23164...,Non-Logged,1,68510c52-5530-4c89-9a8a-0ee36ab61546,1660338410977,0,10000,9.24,1,1660338410977


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577942 entries, 0 to 577941
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype          
---  ------                   --------------   -----          
 0   userId                   577942 non-null  string[pyarrow]
 1   userType                 577942 non-null  string[pyarrow]
 2   historySize              577942 non-null  int64[pyarrow] 
 3   history                  577942 non-null  string[pyarrow]
 4   timestampHistory         577942 non-null  string[pyarrow]
 5   numberOfClicksHistory    577942 non-null  string[pyarrow]
 6   timeOnPageHistory        577942 non-null  string[pyarrow]
 7   scrollPercentageHistory  577942 non-null  string[pyarrow]
 8   pageVisitsCountHistory   577942 non-null  string[pyarrow]
 9   timestampHistory_new     577942 non-null  string[pyarrow]
dtypes: int64[pyarrow](1), string[pyarrow](9)
memory usage: 743.7 MB


Temos 10 colunas diferentes, sendo elas, segundo a própria documentação da FIAP:

- `userId`: id do usuário.
- `userType`: usuário logado ou anônimo.
- `historySize`: quantidade de notícias lidas pelo usuário.
- `history`: lista de notícias visitadas pelo usuário.
- `timestampHistory`: momento em que o usuário visitou a página.
- `timeOnPageHistory`: quantidade de ms em que o usuário ficou na página.
- `numberOfClicksHistory`: quantidade de clicks na matéria.
- `scrollPercentageHistory`: quanto o usuário visualizou da matéria.
- `pageVisitsCountHistory`: quantidade de vezes que o usuário visitou a matéria.

Por mais que não esteja no enunciado do projeto, também existe a coluna `timestampHistory_new` no dataset, mas parece ser um erro de coluna duplicada, já que seu valor é idêntico ao da coluna `timestampHistory`.

In [6]:
display(
    "Há algum valor diferente quando comparamos as `timestampHistory`?",
    (train["timestampHistory"] != train["timestampHistory_new"]).any(),
)

'Há algum valor diferente quando comparamos as `timestampHistory`?'

False

Sendo assim, iremos remover a coluna:

In [7]:
train = train.drop(columns=["timestampHistory_new"])
train.head(1)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
0,acf7fbca426d323f580adddb4fa0f73598294df91cdaee...,Non-Logged,1,d6644c32-5828-4f88-ba44-a8d74c01d7bf,1657303857108,24,48232,47.5,1


**Temos usuários iguais/duplicados no dataset?**\
Dependendo da forma que o G1 lida com os dados, talvez existam usuários com `userId` iguais, mas cada um com um `userType` diferente.

In [8]:
display(train.duplicated().sum())
display(train.duplicated(subset=["userId"]).sum())

np.int64(0)

np.int64(0)

O CSV apenas separa por vírgulas as colunas que são listas. Isso faz com que elas sejam interpretadas pelo tipo "string" quando são lidar pela engine do Pandas com PyArrow. Sendo assim, é bom convertê-las para conseguirmos utilizar métodos de listas nelas nas análises.

In [9]:
list_columns = [
    "history",
    "timestampHistory",
    "numberOfClicksHistory",
    "timeOnPageHistory",
    "scrollPercentageHistory",
    "pageVisitsCountHistory",
]

train[list_columns] = train[list_columns].apply(lambda x: x.str.split(", "))

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577942 entries, 0 to 577941
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype                      
---  ------                   --------------   -----                      
 0   userId                   577942 non-null  string[pyarrow]            
 1   userType                 577942 non-null  string[pyarrow]            
 2   historySize              577942 non-null  int64[pyarrow]             
 3   history                  577942 non-null  list<item: string>[pyarrow]
 4   timestampHistory         577942 non-null  list<item: string>[pyarrow]
 5   numberOfClicksHistory    577942 non-null  list<item: string>[pyarrow]
 6   timeOnPageHistory        577942 non-null  list<item: string>[pyarrow]
 7   scrollPercentageHistory  577942 non-null  list<item: string>[pyarrow]
 8   pageVisitsCountHistory   577942 non-null  list<item: string>[pyarrow]
dtypes: int64[pyarrow](1), list<item: string>[pyarrow](6), string

O usuário só pode estar logado ou deslogado (usuário anônimo/sem uma conta). Com essa informação, futuramente este campo pode ser tratado para virar um inteiro, sendo 1 (logados) e 0 (deslogados).

O dataset está longe de estar balanceado pelo tipo do usuário. Tem-se quase 80% da base com usuários `Non-Loged`, isso já nos mostra que muitas pessoas acessam as notícias sem login ou criação de uma conta.

In [11]:
train["userType"].value_counts(normalize=True) * 100

userType
Non-Logged    79.394818
Logged        20.605182
Name: proportion, dtype: double[pyarrow]

**Todas as listas possuem o mesmo tamanho?**\
Como elas falam sobre o histórico do usuário, o correto é que cada lista em cada linha tenha o mesmo tamanho que aparece na coluna `historySize`.

- Sim! Fizemos uma função que compara o tamanho em `historySize` com o tamanho das listas que temos no dataframe. Ela retornou `True`, indicando que todas as listas possuem o tamanho correto.


In [12]:
def check_size(row: pd.Series, size: int, list_columns: list[str]) -> bool:
    """Check if the len of `historySize` is equal to the len of other columns.

    Args:
        row (pd.Series): the row containing the columns in `list_columns`
        size (int): the size to be checked
        list_columns (list[str]): list with column names of the dataframe

    Returns:
        bool: result of the comparison, returns True if equal.

    """
    return all(len(row[col_list]) == size for col_list in list_columns)


all(train.apply(lambda x: check_size(x, x["historySize"], list_columns), axis=1))

True

**Todos os usuários possuem ao menos um valor em seu histórico de acesso?**\
Isso é relevante para entender quando o G1 começa a monitorar o usuário: é assim que ele entra em qualquer página ou somente quando acessa sua primeira notícia?

Vendo que todos os usuários possuem algum valor em seu `history`, isso já fica claro que é a partir da primeira notícia que o monitoramento começa. Todos os usuários possuem algum histórico, mesmo que seja de apenas uma notícia.

In [13]:
all(train["history"].apply(len) > 0)

True

In [14]:
train["historySize"].describe()

count     577942.0
mean     14.056689
std      46.037793
min            1.0
25%            1.0
50%            2.0
75%            6.0
max         7004.0
Name: historySize, dtype: double[pyarrow]

In [15]:
train["historySize"].median()

2.0

O tamanho do histórico possui 14 de média e 2 de mediana, indicando que alguns usuários possuem um valor de histórico bem alto na base. Os percentis baixos junto com o tamanho máximo alto também indica isso.

Infelizmente, como não há uma coluna indicando a data de início que o usuário começou a ter seu histórico monitorado, não temos como saber se o tamanho do histórico dele é proporcional ao tempo que ele acessa o G1.

In [16]:
BIG_HIST_SIZE = 800
display(
    f"Há {(train['historySize'] >= BIG_HIST_SIZE).sum()} usuários com um histórico \
maior ou igual que {BIG_HIST_SIZE}. Temos {train['userId'].count()} usuários na base."
)

train.sort_values(by="historySize", ascending=False).head(5)

'Há 135 usuários com um histórico maior ou igual que 800. Temos 577942 usuários na base.'

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
433750,1b9bf55cf5c0d7ba34e8925dbbc98da17c5ab360d907c5...,Non-Logged,7004,['26b69894-5e12-4e11-b36a-94e6ec3172a0' '9277...,['1658713338344' '1658713440930' '165871481029...,['0' '20' '1' ... '0' '0' '0'],['136395' '290000' '160971' ... '10000' '10000...,['15.77' '12.72' '4.26' ... '6.19' '6.38' '7.95'],['5' '5' '9' ... '1' '1' '1']
347719,1f7a8e71d6d871e6a77f193c0cece8a6ebcd304dcfddcc...,Logged,5907,['f609464e-5777-4bc9-828a-47f982cc3fed' '007f...,['1656644949511' '1656645019617' '165664533024...,['0' '0' '0' ... '0' '0' '0'],['40000' '28550' '98640' ... '50000' '25196' '...,['43.16' '38.38' '16.89' ... '43.98' '35.02' '...,['1' '1' '5' ... '1' '1' '1']
251786,83a2ba48f07e49f33ef4aa3ceb90933f9adff1ddb6ed16...,Non-Logged,4914,['4745a8d7-9f4f-4beb-8b8d-5adb92000082' '1615...,['1656672728140' '1656672826027' '165667447595...,['0' '0' '0' ... '2' '2' '0'],['10000' '10000' '27069' ... '90000' '40000' '...,['6.88' '6.28' '25.96' ... '44.79' '11.4' '14....,['1' '1' '1' ... '1' '1' '1']
386620,d3b82a1305105bcaaa646d2da1e97b6935deddbcfbd12b...,Logged,3580,['eaea3d49-6fc3-475d-ba30-fa02a91daa98' '4e1d...,['1656645129077' '1656645137042' '165664523380...,['0' '1' '8' ... '17' '1' '0'],['10000' '23204' '100000' ... '110000' '33506'...,['13.3' '27.72' '27.03' ... '40.09' '18.55' '2...,['1' '1' '2' ... '1' '1' '1']
136184,9d176e2202250e7925b17eae0978eee726f8ff9372e45c...,Non-Logged,3318,['07ee94b0-9c39-448e-b96c-3d67f9c1670f' 'fad4...,['1656691004852' '1656691099012' '165669126445...,['24' '55' '8' ... '16' '63' '72'],['89910' '156859' '20274' ... '39313' '114801'...,['72.44' '79.86' '35.65' ... '58.53' '98.72' '...,['1' '1' '1' ... '1' '1' '1']


**Uma mesma notícia pode aparecer mais de uma vez no histórico do usuário?**\
Como o G1 lida com um usuário que acessa uma mesma notícia mais de uma vez?

O usuário abaixo tem um histórico com quatro notícias. Dentre elas, pela coluna `pageVisitsCountHistory` ele acessou a primeira página 12 vezes, a segunda 2 vezes, a terceira 1 vez e a quarta 4 vezes.

Analisando as outras colunas de histórico, chegamos a algumas conclusões:
- O `history` e nem as outras colunas aumentam de tamanho, mesmo ele tendo visto as páginas mais vezes
- O `timestampHistory` segue a ordem do primeiro acesso do usuário, ou seja, o 1° valor da lista é o menor que o segundo, o 2° é menor do que o terceiro, e assim vai até o último
- Tudo indica que o `numberOfClicksHistory` é um somatório dos clicks em cada um dos acessos na página. Um forte indicativo disso é que a 3° página diz que recebeu apenas seis clicks e foi acessada uma vez, mas as outras receberam vários outros clicks.
- Assim como o `numberOfClicksHistory`, o `timeOnPageHistory` parece funcionar da mesma forma, somando o tempo que o usuário passou em cada acesso.
- Diferente dos itens acima, o `scrollPercentageHistory` parece não agir da mesma forma, pois é extremamente improvável que o usuário tenha apenas "20.83%" de acesso na primeira página, sendo que a visitou 12 vezes, passou o maior tempo nesta página do que as outras e deu 127 clicks durante suas visitas.

Também algumas conclusões sobre os tipos:
- `timestampHistory` pode ser convertido para a data real - talvez isso poderia indicar uma certa sazonalidade em algumas notícias. Notícias sobre o Natal, por exemplo, provavelmente são mais acessadas durante os meses de novembro, dezembro e janeiro.
- `timeOnPageHistory` representa, provavelmente, o tempo na página em milissegundos (ms), já que se medirmos esses tempos em segundos ou microsegundos, as medidas ficariam improváveis para um caso real

In [17]:
with pd.option_context("display.max_colwidth", 1):
    display(
        train.query(
            "userId=='6aa5109374f6534b0400ccbbdb9c7b7d64c40a663fe51c95102754e28948f92e'"
        )
    )

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
377947,6aa5109374f6534b0400ccbbdb9c7b7d64c40a663fe51c95102754e28948f92e,Non-Logged,4,['7fe849c0-4a55-429d-b480-11ee216909dd' '2de36771-38b8-4e2c-a2b7-d00590dd9009' 'befa491a-1c4a-43b8-ad4e-8bfabe03320e' 'c7c70809-d617-40a3-99a0-a95c87679acd'],['1658259718982' '1658424852871' '1658859110243' '1660064956101'],['127' '155' '6' '78'],['350199' '320000' '39671' '151452'],['20.83' '80.94' '45.18' '22.32'],['12' '2' '1' '4']


In [18]:
# Comparison of timestampHistory
(1658259718982 < 1658424852871) and (1658424852871 < 1658859110243) and (  # noqa: PLR0133
    1658859110243 < 1660064956101  # noqa: PLR0133
)

True

As unidades devem estar em milissegundos (ms), pois é a unidade que mais faz sentido:

In [19]:
pd.to_datetime([1658259718982, 1658424852871, 1658859110243, 1660064956101], unit="ms")

DatetimeIndex(['2022-07-19 19:41:58.982000', '2022-07-21 17:34:12.871000',
               '2022-07-26 18:11:50.243000', '2022-08-09 17:09:16.101000'],
              dtype='datetime64[ns]', freq=None)

## Análise dos dados de validação

In [21]:
validation_path = Path("../data/raw/validacao.csv")
validation = read_csv_pyarrow(validation_path)
validation.head(2)

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]


A primeira coisa que notamos no dataset é a **falta de colunas em relação aos dados de treino**. Tem-se apenas quatro colunas, que significam a mesma coisa do dataset de treino:

- `userId`: id do usuário.
- `userType`: usuário logado ou anônimo.
- `history`: lista de notícias visitadas pelo usuário.
- `timestampHistory`: momento em que o usuário visitou a página.

In [22]:
validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112184 entries, 0 to 112183
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype          
---  ------            --------------   -----          
 0   userId            112184 non-null  string[pyarrow]
 1   userType          112184 non-null  string[pyarrow]
 2   history           112184 non-null  string[pyarrow]
 3   timestampHistory  112184 non-null  string[pyarrow]
dtypes: string[pyarrow](4)
memory usage: 19.3 MB


Também não temos dados duplicados

In [23]:
display(validation.duplicated().sum())
display(validation.duplicated(subset=["userId"]).sum())

np.int64(0)

np.int64(0)

Diferente do dataset de treino, no de validação temos um dataset mais balanceado pelo tipo de usuário.

Novamente, isso não importa tanto assim, já que o objetivo do projeto visa atender todos os tipos de usuários igualmente: recomendando as notícias mais apropriadas.

In [24]:
validation["userType"].value_counts(normalize=True) * 100

userType
Non-Logged    58.176745
Logged        41.823255
Name: proportion, dtype: double[pyarrow]

Novamente, por conta da forma que os dados são lidos, precisamos transformar as colunas para uma lista de strings.

Obs.: foi necessário tratar melhor os dados, pois eles possuem uma formatação um pouco diferente do que tinham no arquivo de treino

In [25]:
list_columns = ["history", "timestampHistory"]

validation["history"] = (
    validation["history"]
    .str.replace(r"[\[\]']", "", regex=True)
    .str.replace(r"\n", "", regex=True)
    .str.split()
)
validation["timestampHistory"] = (
    validation["timestampHistory"].str.replace(r"[\[\]]", "", regex=True).str.split()
)
validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112184 entries, 0 to 112183
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype                      
---  ------            --------------   -----                      
 0   userId            112184 non-null  string[pyarrow]            
 1   userType          112184 non-null  string[pyarrow]            
 2   history           112184 non-null  list<item: string>[pyarrow]
 3   timestampHistory  112184 non-null  list<item: string>[pyarrow]
dtypes: list<item: string>[pyarrow](2), string[pyarrow](2)
memory usage: 19.8 MB


E, igual com o dataset de treino, as colunas de histórico possuem o mesmo tamanho - o que é o correto.

> Como não temos a coluna `historySize`, aqui apenas comparamos os tamanhos das duas colunas

In [26]:
display(validation.head(2))
display("Tamanho de history:", validation["history"].head(2).apply(len))

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,['1660533136590' '1660672113513']
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],['1660556860253']


'Tamanho de history:'

0    2
1    1
Name: history, dtype: int64

In [27]:
all(validation["history"].apply(len) == validation["timestampHistory"].apply(len))

True

Quando verificamos os IDs da tabela de validação dentro da tabela de treino, vemos que eles existem nas duas tabelas, porém, na de treino, a informação é mais completa do que na de validação.

In [28]:
all(validation["userId"].isin(train["userId"]))

True

In [29]:
display(f"Primeiro userId na validation: f{validation['userId'][0]}")

with pd.option_context("display.max_colwidth", 1):
    display("Validação:", validation.head(1))
    display("Treino:", train.query(f"userId == '{validation['userId'][0]}'"))

'Primeiro userId na validation: fe25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9'

'Validação:'

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c59ff6-fb82-4258-918f-2910cb2d4c52'],['1660533136590' '1660672113513']


'Treino:'

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
482774,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9,Logged,6,['c7b6f56c-3304-4fe5-994a-50c8937d6431' 'c6af277b-d658-493c-9d7e-342e5d6759c4' '064b539b-a4cd-4781-b97f-9e9859ce15f7' 'bf257382-74fb-4392-ad6a-143240e39f81' 'f774e860-8103-49bf-b04e-63b316735e4b' 'c4a64d0d-09b8-4f82-8e32-4d2ec6067226'],['1657003078783' '1659484116867' '1659700766244' '1659929500885' '1660062239512' '1660182601797'],['5' '1' '10' '16' '4' '2'],['20848' '10000' '37197' '90813' '23755' '100000'],['36.51' '19.78' '46.59' '36.45' '32.46' '13.89'],['1' '1' '1' '1' '1' '1']


In [30]:
pd.to_datetime(
    [
        1660533136590,  # validação
        1660672113513,  # validação
        1657003078783,  # treino
        1659484116867,  # treino
        1659700766244,  # treino
        1659929500885,  # treino
        1660062239512,  # treino
        1660182601797,  # treino
    ],
    unit="ms",
)

DatetimeIndex(['2022-08-15 03:12:16.590000', '2022-08-16 17:48:33.513000',
               '2022-07-05 06:37:58.783000', '2022-08-02 23:48:36.867000',
               '2022-08-05 11:59:26.244000', '2022-08-08 03:31:40.885000',
               '2022-08-09 16:23:59.512000', '2022-08-11 01:50:01.797000'],
              dtype='datetime64[ns]', freq=None)

In [31]:
with pd.option_context("display.max_colwidth", 1):
    display("Validação:", validation.head(2))
    display("Treino:", train.query(f"userId == '{validation['userId'][1]}'"))

'Validação:'

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c59ff6-fb82-4258-918f-2910cb2d4c52'],['1660533136590' '1660672113513']
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdeee870d49b897e1e99cd,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],['1660556860253']


'Treino:'

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
554897,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdeee870d49b897e1e99cd,Logged,1,['9a37393c-15fc-413a-b285-860da171d0a2'],['1656668214243'],['15'],['30883'],['35.35'],['1']


In [32]:
# validação e treino
pd.to_datetime([1660556860253, 1656668214243], unit="ms")

DatetimeIndex(['2022-08-15 09:47:40.253000', '2022-07-01 09:36:54.243000'], dtype='datetime64[ns]', freq=None)

Pela análise acima, percebe-se algo estranho:
- O `history` **não contém** as mesmas notícias nos dois datasets, mas é o mesmo usuário.
- O `timestampHistory` segue da mesma forma, não registrou as notícias corretamente.

Retirando dúvidas com a FIAP e analisando melhor os datasets, percebemos que **é o mesmo usuário**, porém com seu histórico de notícias sendo capturado em um período diferente.

Exemplo com o usuário de ID "d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdeee870d49b897e1e99cd":
- No dataset de treino, ele possui uma notícia em seu histórico. Ela foi acessada na data 2022-07-01
- Já no dataset de validação, ele possui uma notícia também, mas ela foi acessada na data 2022-08-15

O mesmo ocorre para outros usuários. Isso nos mostra que o dataset de treino captura as notícias **mais antigas**, enquanto o de validação irá capturar as **mais recentes** consumidas pelo usuário.

Sendo assim, a forma correta de validar nosso modelo será:
1. Buscamos no dataset de treino todas as linhas que contém os IDs que estão no dataset de validação
2. Com essas linhas (do dataset de treino), iremos usar nosso modelo para recomendar/prever as N próximas notícias que esses usuários devem consumir
   1. O número de previsões (N) pode ser fixo, mas é melhor que seja dinâmico, onde o valor de N irá ser definido pelo tamanho do histórico do usário no dataset de validação. Ou seja, se o usuário tem um `history` de três notícias no dataset de validação, o valor de N deve ser 3.
3. Iremos pegar as notícias previstas e comparar com as que estão no dataset de validação
4. Com essa comparação, podemos criar algum tipo de score (ainda será definido) para validar se o modelo está prevendo bem ou não.

## Análise dos itens complementares

In [33]:
itens_path = "../data/raw/itens/itens/*.csv"
itens_files = Path.cwd().glob(itens_path)

itens = pd.concat(
    (read_csv_pyarrow(file) for file in itens_files),
    ignore_index=True,
)

itens.head(2)

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém Ádrio Denner/ AD Produ...,As expressões santarenas não significam apenas...


In [34]:
itens["title"][255598]

'Eleições: título de eleitor cancelado, o que fazer? Saiba como regularizar a tempo de votar'

Para os itens, a URL nos dá uma certa informação sobre o tipo da notícia. Ex.:
- http://g1.globo.com/am/amazonas/noticia/2022/06/18/(...)
  - A URL fornece a data que a notícia foi postada (18/06/2022)
  - A URL fornece as "tags" da notícia:
    - É do Amazonas, por isso contém 'am'
    - A notícia é específica de Amazonas, por isso contém "amazonas"
    - É uma notícia, então fica com "noticia" na tag
- A URL "http://g1.globo.com/pa/santarem-regiao/noticia/2019/06/20/(...)" funciona de forma igual ao exemplo acima
- http://g1.globo.com/mundo/noticia/2022/07/08/
  - A notícia é internacional, então recebe a tag "mundo"
  - Diferente das outras, não teve uma tag específica para a região que ocorreu. Isso já indica uma certa probabilidade de que o G1 recomende notícias para as pessoas baseadas na sua localização, infelizmente é um dado que não temos
- http://g1.globo.com/politica/noticia/2021/09/09/(...)
  - Aqui a tag mudou, por mais que seja algo brasileiro, apenas ficou como "politica"
- http://g1.globo.com/politica/eleicoes/2022/noticia/2022/04/18/(...)
  - Aqui a tag ficou mais específica, ainda está dentro de "politica", mas faz parte das eleições
  - O "2022" aparece duas vezes, mas tem significados diferentes, um deles indica que a notícia é sobre as eleições de 2022, não necessariamente fala sobre a data de publicação da notícia

Dito isso, temos o seguinte padrão:
1. Aparece a URL base do G1 (http://g1.globo.com/)
2. Aparece uma longa URL (tamanho variado) que fornece tags sobre a notícia, como sua região (estado, cidade ou se é internacional) ou seu tema (política, eleições, economia, etc.)
3. Aparece a data da postagem da notícia - que é o mesmo valor que aparece na coluna `issued`
4. E, por fim, tem-se a página e sua extensão (ghtml, por exemplo). Essa parte pode servir como a coluna `title` já tratada, pois segue o mesmo padrão, mas sem acentos. Ex.:
   - Na url: "eleicoes-titulo-de-eleitor-cancelado-o-que-fazer-saiba-como-regularizar-a-tempo-de-votar.ghtml"
   - No title: "Eleições: título de eleitor cancelado, o que fazer? Saiba como regularizar a tempo de votar"
   - Com isso, pode-se utilizar a URL para pular a etapa de tratamento do título, pois a URL já aplica:
     - Remoção de caracteres especiais (pontos, vírgulas etc.)
     - Substituição de acentos pelo caractere de origem: "ç" vira "c", "í" vira "i"
     - Tudo fica em letras minúsculas

Agora, sobre as outras colunas, a mais preocupante parece a `body` - o corpo da notícia.

Ela contém um enorme texto e precisará de uma série de tratamentos para ser processada corretamente em algoritmos de machine learning.

In [36]:
itens["title"][0]

'Caso Bruno e Dom: 3º suspeito tem prisão temporária decretada pela Justiça do AM'

In [37]:
itens["caption"][0]

'Jeferson da Silva Lima foi escoltado por agentes da Polícia Federal ao Fórum de Justiça do município para a audiência de custódia'

In [35]:
itens["body"][0]

'Após audiência de custódia, a Justiça do Amazonas decretou, na tarde deste sábado (18), a prisão temporária,  por 30 dias, de Jeferson da Silva Lima, conhecido como "Pelado da Dinha". Ele teve participação direta na morte do indigenista Bruno Pereira e do jornalista inglês Dom Phillips, aponta as investigações. \n"Pelado da Dinha" foi considerado foragido na noite de sexta-feira (17) após ter o mandado de prisão expedido e não ser localizado pelas autoridades. Ele se entregou na delegacia de Atalaia do Norte, a 1.136 quilômetros de Manaus, nas primeiras horas da manhã deste sábado, onde foi ouvido pelo delegado Alex Perez Timóteo. \n"Pelado da Dinha" foi considerado foragido na noite de sexta-feira (17) após ter o mandado de prisão expedido \nRôney Elias/Rede Amazônica \nDurante a tarde, Jeferson foi escoltado por agentes da Polícia Federal ao Fórum de Justiça do município para a audiência de custódia e teve a prisão temporária decretada. \nPerícia confirma identificação dos restos mo